In [1]:
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/18 09:12:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
df = spark.read \
    .option("header", "true") \
    .csv('title.basics.tsv.gz', sep='\t')

In [6]:
df.count()

9759339

In [7]:
df.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [9]:
df.select('primaryTitle', 'originalTitle').show()

+--------------------+--------------------+
|        primaryTitle|       originalTitle|
+--------------------+--------------------+
|          Carmencita|          Carmencita|
|Le clown et ses c...|Le clown et ses c...|
|      Pauvre Pierrot|      Pauvre Pierrot|
|         Un bon bock|         Un bon bock|
|    Blacksmith Scene|    Blacksmith Scene|
|   Chinese Opium Den|   Chinese Opium Den|
|Corbett and Court...|Corbett and Court...|
|Edison Kinetoscop...|Edison Kinetoscop...|
|          Miss Jerry|          Miss Jerry|
| Leaving the Factory|La sortie de l'us...|
|Akrobatisches Pot...|Akrobatisches Pot...|
|The Arrival of a ...|L'arrivée d'un tr...|
|The Photographica...|Le débarquement d...|
| The Waterer Watered|   L'arroseur arrosé|
| Autour d'une cabine| Autour d'une cabine|
|Boat Leaving the ...|Barque sortant du...|
|Italienischer Bau...|Italienischer Bau...|
|Das boxende Känguruh|Das boxende Känguruh|
|    The Clown Barber|    The Clown Barber|
|      The Derby 1895|      The 

In [7]:
'''
schema = {
    tconst = Column(String), # ID
    titleType = Column(String), # the type/format of the title
    primaryTitle = Column(String), # more popular title
    originalTitle = Column(String),
    isAdult = Column(Integer), # 0: non-adult title; 1: adult title
    startYear = Column(Integer),
    endYear = Column(Integer),
    runTimeMinutes = Column(Integer),
    genres = List(Column(String)) # additional table?
'''

'\nschema = {\n    tconst = Column(String), # ID\n    titleType = Column(String), # the type/format of the title\n    primaryTitle = Column(String), # more popular title\n    originalTitle = Column(String),\n    isAdult = Column(Integer), # 0: non-adult title; 1: adult title\n    startYear = Column(Integer),\n    endYear = Column(Integer),\n    runTimeMinutes = Column(Integer),\n    genres = List(Column(String)) # additional table?\n'

In [13]:
df.createOrReplaceTempView("titles")

sql_df = spark.sql("SELECT * FROM titles LIMIT 10")
sql_df.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [25]:
df.select('isAdult').distinct().show()

+-------+
|isAdult|
+-------+
|   2020|
|   2019|
|   2017|
|   2014|
|   2005|
|      0|
|     \N|
|   1981|
|   2022|
|   2023|
|      1|
+-------+



In [20]:
from pyspark.sql import types
title_schema = types.StructType([
    types.StructField('tconst', types.StringType(), True),
    types.StructField('titleType', types.StringType(), True),
    types.StructField('primaryTitle', types.StringType(), True),
    types.StructField('originalTitle', types.StringType(), True),
    types.StructField('isAdult', types.IntegerType(), True),
    types.StructField('startYear', types.IntegerType(), True),
    types.StructField('endYear', types.IntegerType(), True),
    types.StructField('runTimeMinutes', types.IntegerType(), True),
    types.StructField('genres', types.StringType(), True)
])

In [35]:
df = spark.read \
    .option("header", "true") \
    .csv(
        'title.basics.tsv.gz', 
        sep='\t',
        schema=title_schema
    )

In [28]:
df.schema['isAdult'].dataType

IntegerType()

In [38]:
df1.select('isAdult').distinct().show()

+-------+
|isAdult|
+-------+
|   null|
|      1|
|      0|
+-------+



In [37]:
from pyspark.sql.functions import when
df1 = df.withColumn('isAdult', when(df.isAdult > 1, None) \
                    .otherwise(df.isAdult))